In [1]:
import numpy as np
from pathlib import Path, PosixPath
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import xarray as xr

Path to dataset

In [5]:
path = PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS')

In [6]:
path

PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS')

# load characteristics 

In [5]:
indicator_files = list(path.glob('*.txt'))

In [6]:
df_clim_indicators = pd.read_csv(indicator_files[0],delimiter=";")
df_geol_indicators = pd.read_csv(indicator_files[1],delimiter=";")
df_hydro_indicators = pd.read_csv(indicator_files[2],delimiter=";")
df_catchment_names = pd.read_csv(indicator_files[3],delimiter=";")
df_soil_indicators = pd.read_csv(indicator_files[4],delimiter=";")
df_topo_indicators = pd.read_csv(indicator_files[5],delimiter=";")
df_vege_indicators = pd.read_csv(indicator_files[6],delimiter=";")
df_vege_indicators['dom_land_cover'] = df_vege_indicators['dom_land_cover'].apply(lambda x: x.strip())

In [7]:
df_char_lst = [df_geol_indicators,
                df_clim_indicators,
                df_hydro_indicators,
                df_catchment_names,
                df_soil_indicators,
                df_vege_indicators, 
                df_topo_indicators]
for index, df in enumerate(df_char_lst):
    df_char_lst[index] = df.set_index('gauge_id')

In [8]:
characteristics_all = pd.concat(df_char_lst,axis=1)

## Forcing path

In [9]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'

In [10]:
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source

In [11]:
sub_dirs_forcing = list(forcing_path.glob("*"))

In [12]:
basin_mean_forcing_files = list(sub_dirs_forcing[0].glob("*.txt"))

In [13]:
basin_mean_forcing_file = basin_mean_forcing_files[0]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name

In [14]:
catchment_char = characteristics_all.loc[int(basin_id)].copy()

In [15]:
len(list(characteristics_all.index))

671

## retrieve alpha

retrieve alpha: not always 1.26

In [15]:
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'

In [16]:
model_ouput_path

PosixPath('/media/davidhaasnoot/files/large_data_sets/CAMELS/basin_timeseries_v1p2_modelOutput_daymet/model_output_daymet/model_output/flow_timeseries/daymet')

In [17]:
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))

In [18]:
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))

In [19]:
alpha = np.array(lst_alpha).mean()

## add streamflow

In [20]:
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

# Combine all 

In [21]:
from read_camels import from_camels_txt, crop_ds, calc_pet

In [22]:
start_date = "1980-01-01T00:00:00Z"
end_date = "2014-12-31T00:00:00Z"

In [23]:
out_path = path.cwd() / "Output" / source
out_path.mkdir(exist_ok=True)

In [24]:
ds0 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path)

## test merge

In [25]:
source_lst = ['daymet','maurer','nldas']
source = source_lst[0]
# Source dependant
forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
forcing_sub_path_2 = f'basin_dataset_public_v1p2'
forcing_sub_path_3 = f'basin_mean_forcing'
forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
sub_dirs_forcing = list(forcing_path.glob("*"))
model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'

In [26]:
# basin dependant
basin_mean_forcing_file = basin_mean_forcing_files[1]
basin_id = basin_mean_forcing_file.name[:8]
folder_id = basin_mean_forcing_file.parent.name
catchment_char = characteristics_all.loc[int(basin_id)].copy()
parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
line_start = "pet_coef"
lst_alpha = []
for file in parameter_files_path:
    with open(file,'r') as lines:
        for line in lines:
            if line[:len(line_start)] == line_start:
                lst_alpha.append(float(line.strip().split("\t")[-1]))
alpha = np.array(lst_alpha).mean()

streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 

In [27]:
ds1 = from_camels_txt(basin_mean_forcing_file, basin_id, source, start_date, end_date, out_path, alpha, catchment_char, streamflow_file_path)

In [28]:
xr.merge([ds0,ds1])

<xarray.Dataset> Size: 2MB
Dimensions:               (time: 12784, data_source: 1, basin_id: 2)
Coordinates:
  * time                  (time) datetime64[ns] 102kB 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 64B b'daymet'
  * basin_id              (basin_id) |S64 128B b'01013500' b'01134500'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 205kB 3.017e+...
    pr                    (basin_id, data_source, time) float64 205kB 0.0 ......
    srad                  (basin_id, data_source, time) float64 205kB 153.4 ....
    swe                   (basin_id, data_source, time) float64 205kB 0.0 ......
    tasmax                (basin_id, data_source, time) float64 205kB -6.54 ....
    tasmin                (basin_id, data_source, time) float64 205kB -16.3 ....
    ...                    ...
    gauge_lat             (basin_id) float64 16B 47.24 44.51
    gauge_lon             (basin_id) float64 16B -68.58 -71.84
    elev_mean             (basin_id) float64 16B 250.3 450.5
    slope_mean            (basin_id) float64 16B 21.64 47.54
    area_gages2           (basin_id) float64 16B 2.253e+03 195.1
    area_geospa_fabric    (basin_id) float64 16B 2.304e+03 209.5
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2617
    lat:              44.49
    elevation(m):     529.0
    area basin(m^2):  194683851.0

# loop

not found 1150900, 6775500, 6846500 in `catchment_char = characteristics_all.loc[int(basin_id)].copy()`

error in 02108000, 05120500, 07067000, 09492400 wrong header daymet

In [29]:
skip_ids = [2081113, 3448942, 9535100, 1150900, 6775500, 6846500]

In [30]:
df

,gauge_id,gauge_lat,gauge_lon,elev_mean,slope_mean,area_gages2,area_geospa_fabric
0,1013500,47.23739,-68.58264,250.31,21.64152,2252.70,2303.95
1,1022500,44.60797,-67.93524,92.68,17.79072,573.60,620.38
2,1030500,45.50097,-68.30596,143.80,12.79195,3676.17,3676.09
3,1031500,45.17501,-69.31470,247.80,29.56035,769.05,766.53
4,1047000,44.86920,-69.95510,310.38,49.92122,909.10,904.94
...,...,...,...,...,...,...,...
666,14309500,42.80400,-123.61091,709.83,110.42527,224.92,226.31
667,14316700,43.34984,-122.72894,952.26,119.08920,587.90,588.01
668,14325000,42.89150,-124.07065,656.53,124.96889,443.07,444.92
669,14362250,42.15401,-123.07532,875.67,109.93127,41.42,43.88


In [36]:
source_lst = ['daymet','maurer','nldas']
for source in source_lst[1:]:
    # Source dependant
    out_path = path.cwd() / "Output" / source
    out_path.mkdir(exist_ok=True)
    forcing_sub_path_1 = f'basin_timeseries_v1p2_metForcing_obsFlow'
    forcing_sub_path_2 = f'basin_dataset_public_v1p2'
    forcing_sub_path_3 = f'basin_mean_forcing'
    forcing_path = path / forcing_sub_path_1 / forcing_sub_path_2 / forcing_sub_path_3 / source
    sub_dirs_forcing = list(forcing_path.glob("*"))
    model_ouput_path = path / f'basin_timeseries_v1p2_modelOutput_{source}' / f'model_output_{source}' / 'model_output' /'flow_timeseries'/ f'{source}'
    streamflow_path = path / forcing_sub_path_1 / forcing_sub_path_2 / 'usgs_streamflow'
    
    for sub_dirs in sub_dirs_forcing:
        basin_mean_forcing_files = list(sub_dirs.glob("*.txt"))
        for basin_mean_forcing_file in basin_mean_forcing_files:
            basin_id = basin_mean_forcing_file.name[:8]
            if int(basin_id) in skip_ids:
                pass 
            else:
                folder_id = basin_mean_forcing_file.parent.name
                catchment_char = characteristics_all.loc[int(basin_id)].copy()
                parameter_files_path = list((model_ouput_path / f'{folder_id}').glob(f'{basin_id}_*_model_parameters.txt'))
                
                if source == "daymet":
                    line_start = "pet_coef"
                elif source != 'daymet':
                    line_start = "PT_COEF"
                lst_alpha = []    
                for file in parameter_files_path:
                    with open(file,'r') as lines:
                        for line in lines:
                            if line[:len(line_start)] == line_start:
                                lst_alpha.append(float(line.strip().split("\t")[-1]))
                alpha = np.array(lst_alpha).mean()
                
                streamflow_file_path = streamflow_path / f'{folder_id}' / f'{basin_id}_streamflow_qc.txt' 
                ds = from_camels_txt(basin_mean_forcing_file, 
                                     basin_id, 
                                     source, 
                                     start_date, 
                                     end_date, 
                                     out_path, 
                                     alpha, 
                                     catchment_char, 
                                     streamflow_file_path)
                ds.close()

In [4]:
files_daymet = list((Path.cwd() / "Output" / "daymet").glob("*nc"))
files_maurer = list((Path.cwd() / "Output" / "maurer").glob("*nc"))
files_nldas = list((Path.cwd() / "Output" / "nldas").glob("*nc"))

In [9]:
file_lst = []
names = files_daymet# + files_maurer + files_nldas
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 10,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [11]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [24]:
time_vars = list(ds.data_vars)[:10]

In [9]:
# from https://gist.github.com/BSchilperoort/256751fe2ea060c50b103f72026590a2
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars}  # don't encode string-based vars!
#     output_file = path_output / f"{dataset}.nc"
#     print(f"Exporting {output_file}!")
#     ds.to_netcdf(output_file, encoding=encoding)

In [ ]:
ds.to_netcdf("combined_daymet.nc", encoding=encoding)

In [ ]:
ds.close()

In [5]:
file_lst = []
names = files_maurer # + files_nldas + files_daymet
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 10,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [6]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [7]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [8]:
ds.to_netcdf("combined_maurer.nc", encoding=encoding)

In [9]:
ds.close()

In [10]:
file_lst = []
names =  files_nldas # + files_daymet + files_maurer
for file in names:
    basin_id = file.name[19:27]
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 10,'data_source':1 })
    ds = ds.expand_dims(dim={'basin_id':np.array([(str('camels_'+ basin_id)).encode()],dtype=np.dtype('|S64'))})
    file_lst.append(ds)

In [11]:
ds = xr.concat(file_lst,dim=('basin_id'))

In [12]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [13]:
ds.to_netcdf("combined_nldas.nc", encoding=encoding)

## combine the three

In [3]:
files = ["combined_nldas.nc","combined_maurer.nc","combined_daymet.nc"]
file_lst = []
for file in files:
    ds = xr.open_dataset(file, chunks={'time': 1000, 'basin_id': 100,'data_source':1 })
    file_lst.append(ds)

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dime

In [4]:
file_lst[0]

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 1)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    gauge_lon             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    elev_mean             (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    slope_mean            (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    area_gages2           (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    area_geospa_fabric    (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2621
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

In [5]:
ds = xr.concat(file_lst,dim=('data_source'))

In [6]:
ds

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    gauge_lon             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    elev_mean             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    slope_mean            (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_gages2           (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_geospa_fabric    (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2621
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

In [7]:
time_vars = list(ds.data_vars)[:10]
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in time_vars} 

In [8]:
ds.to_netcdf("combined_CAMELS.nc", encoding=encoding)

In [10]:
ds = xr.open_dataset("combined_CAMELS.nc")

In [11]:
ds

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 ...
    pr                    (basin_id, data_source, time) float64 ...
    srad                  (basin_id, data_source, time) float64 ...
    swe                   (basin_id, data_source, time) float64 ...
    tasmax                (basin_id, data_source, time) float64 ...
    tasmin                (basin_id, data_source, time) float64 ...
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 ...
    gauge_lon             (data_source, basin_id) float64 ...
    elev_mean             (data_source, basin_id) float64 ...
    slope_mean            (data_source, basin_id) float64 ...
    area_gages2           (data_source, basin_id) float64 ...
    area_geospa_fabric    (data_source, basin_id) float64 ...
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2621
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

#  Combine with caravan

In [2]:
ds_caravan = xr.open_dataset("camels_CARAVAN.nc", chunks={'time': 1000, 'basin_id': 100,'data_source':1 })

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [3]:
ds_caravan

<xarray.Dataset>
Dimensions:                             (basin_id: 482, time: 14609)
Coordinates:
  * time                                (time) datetime64[ns] 1981-01-01 ... ...
  * basin_id                            (basin_id) <U15 'camels_01022500' ......
Data variables: (12/55)
    snow_depth_water_equivalent_mean    (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    surface_net_solar_radiation_mean    (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    surface_pressure_mean               (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    temperature_2m_mean                 (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (basin_id, time) float32 dask.array<chunksize=(100, 1000), meta=np.ndarray>
    ...                                  ...
    moisture_index                      (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    seasonality                         (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    high_prec_freq                      (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    high_prec_dur                       (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    low_prec_freq                       (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    low_prec_dur                        (basin_id) float64 dask.array<chunksize=(100,), meta=np.ndarray>
Attributes:
    history:  Merged together from separate files; All forcing and state vari...

need to add data source: era5-land

In [4]:
ds_caravan = ds_caravan.expand_dims(dim={'data_source':np.array([(str('era5_land')).encode()],dtype=np.dtype('|S64'))})

In [5]:
ds_caravan

<xarray.Dataset>
Dimensions:                             (data_source: 1, basin_id: 482,
                                         time: 14609)
Coordinates:
  * data_source                         (data_source) object b'era5_land'
  * time                                (time) datetime64[ns] 1981-01-01 ... ...
  * basin_id                            (basin_id) <U15 'camels_01022500' ......
Data variables: (12/55)
    snow_depth_water_equivalent_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    surface_net_solar_radiation_mean    (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    surface_net_thermal_radiation_mean  (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    surface_pressure_mean               (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    temperature_2m_mean                 (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    dewpoint_temperature_2m_mean        (data_source, basin_id, time) float32 dask.array<chunksize=(1, 100, 1000), meta=np.ndarray>
    ...                                  ...
    moisture_index                      (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    seasonality                         (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    high_prec_freq                      (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    high_prec_dur                       (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    low_prec_freq                       (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    low_prec_dur                        (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
Attributes:
    history:  Merged together from separate files; All forcing and state vari...

In [6]:
ds_camels = xr.open_dataset("combined_CAMELS.nc", chunks={'time': 1000, 'basin_id': 100,'data_source':1 })

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "basin_id" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\xarray\core\dataset.py:275: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [7]:
ds_camels

<xarray.Dataset>
Dimensions:               (basin_id: 671, time: 12784, data_source: 3)
Coordinates:
  * basin_id              (basin_id) |S15 b'camels_01013500' ... b'camels_144...
  * time                  (time) datetime64[ns] 1980-01-01 ... 2014-12-31
  * data_source           (data_source) |S64 b'nldas' b'maurer' b'daymet'
Data variables: (12/69)
    dayl                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    pr                    (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    srad                  (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    swe                   (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmax                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    tasmin                (basin_id, data_source, time) float64 dask.array<chunksize=(100, 1, 1000), meta=np.ndarray>
    ...                    ...
    gauge_lat             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    gauge_lon             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    elev_mean             (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    slope_mean            (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_gages2           (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
    area_geospa_fabric    (data_source, basin_id) float64 dask.array<chunksize=(1, 100), meta=np.ndarray>
Attributes:
    title:            Basin mean forcing data
    history:          Created by David Haasnoot for eWatercycle using CAMELS ...
    data_source:      CAMELS was compiled by A. Newman et al. `A large-sample...
    url_source_data:  https://dx.doi.org/10.5065/D6MW2F4D
    units:            daylight(s), precipitation(mm/day), mean radiation(W/m2...
    alpha:            1.2621
    lat:              46.84
    elevation(m):     353.0
    area basin(m^2):  2260093113.0

In [8]:
ds_out = xr.concat([ds_caravan, ds_camels], dim='data_source')

C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anaconda3\envs\uni\Lib\site-packages\dask\array\core.py:4836: PerformanceWarning: Increasing number of chunks by factor of 13
  result = blockwise(
C:\Users\david\anacond

In [44]:
lst_float_vars = []
for var in ds_out.data_vars:
    if ds_out.data_vars[var].dtype == np.dtype(float) or ds_out.data_vars[var].dtype == np.dtype('float32'):
        lst_float_vars.append(var)


In [45]:
compression = {'zlib': True,'complevel': 8}
encoding = {var: compression for var in lst_float_vars} 

In [ ]:
ds_out.to_netcdf("camels_updated.nc", encoding=encoding)